In [1]:
import tensorflow
print(tensorflow.__version__)

1.0.1


In [2]:
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [9]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

In [31]:
housing.data.shape

(20640, 8)

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  2.75443
Epoch 100 MSE =  0.632222
Epoch 200 MSE =  0.57278
Epoch 300 MSE =  0.558501
Epoch 400 MSE =  0.549069
Epoch 500 MSE =  0.542288
Epoch 600 MSE =  0.537379
Epoch 700 MSE =  0.533822
Epoch 800 MSE =  0.531243
Epoch 900 MSE =  0.529371


In [33]:
best_theta

array([[  2.06855226e+00],
       [  7.74078071e-01],
       [  1.31192386e-01],
       [ -1.17845096e-01],
       [  1.64778158e-01],
       [  7.44080753e-04],
       [ -3.91945168e-02],
       [ -8.61356616e-01],
       [ -8.23479712e-01]], dtype=float32)

In [34]:
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error), name="mes")

In [38]:
optimizer = tf.train.MomentumOptimizer(learning_rate= learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init= tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print("Best theta:", best_theta)

Epoch 0 MSE =  2.75443
Epoch 100 MSE =  0.527316
Epoch 200 MSE =  0.524414
Epoch 300 MSE =  0.524328
Epoch 400 MSE =  0.524322
Epoch 500 MSE =  0.524321
Epoch 600 MSE =  0.524321
Epoch 700 MSE =  0.52432
Epoch 800 MSE =  0.524321
Epoch 900 MSE =  0.524321
Best theta: [[ 2.06855798]
 [ 0.82961673]
 [ 0.11875112]
 [-0.26552212]
 [ 0.30569226]
 [-0.00450316]
 [-0.03932616]
 [-0.89989167]
 [-0.87054664]]
